In [27]:
import os
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import json
import numpy as np



In [28]:
def extraccion_actual(fecha, CLIENT_ID, manana):

    url="https://api.seatgeek.com/2/events"
    params = {
    "client_id": CLIENT_ID,
    "venue.city": "New York",   
    "sort": "score.desc",        
    "per_page": 100,            
    "datetime_local.gte": fecha,
    "datetime_local.lte": manana,
    }
    
    response = requests.get(url, params=params)
    assert response.status_code == 200, "Error en la extracción de eventos"

    return response.json()

    
    

In [29]:
fecha_hoy_obj = datetime.now() 
manana_obj = fecha_hoy_obj + timedelta(days = 1)

fecha_hoy_str = fecha_hoy_obj.strftime('%Y-%m-%d')
manana_str = manana_obj.strftime('%Y-%m-%d')


API_KEY = os.getenv('CLIENT_ID_SEATGEEK')
assert API_KEY is not None, "Falta la variable de entorno CLIENT_ID_SEATGEEK"


data = extraccion_actual(fecha_hoy_str, API_KEY, manana_str)


In [30]:
data

{'events': [{'access_method': None,
   'announce_date': '2025-07-15T00:00:00',
   'announcements': {},
   'conditional': False,
   'contingent': False,
   'created_at': '2025-07-15T13:42:42',
   'date_tbd': False,
   'datetime_local': '2026-02-16T19:00:00',
   'datetime_tbd': False,
   'datetime_utc': '2026-02-17T00:00:00',
   'description': '',
   'onsale': None,
   'enddatetime_utc': '2026-02-17T00:00:00',
   'event_promotion': None,
   'game_number': 0,
   'home_game_number': 0,
   'id': 17624799,
   'integrated': None,
   'is_open': False,
   'is_visible': True,
   'is_visible_override': 'UNSET',
   'links': [{'id': '158924366',
     'url': '',
     'provider': 'stubhub',
     'primary': False,
     'display_name': '',
     'logos': {},
     'sales': [],
     'link_type': 'ID'},
    {'id': '3430913',
     'url': '',
     'provider': 'rapidseats',
     'primary': False,
     'display_name': '',
     'logos': {},
     'sales': [],
     'link_type': 'ID'},
    {'id': 'Z1r9uZrrZbpZ1Avp

In [31]:
eventos_limpios = []

for e in data['events']:
    info = {
        'nombre_evento': e.get('title'),
        'tipo': e.get('type'),
        'hora_inicio': e.get('datetime_local'),
        'lugar': e['venue'].get('name'),
        'direccion': e['venue'].get('address', 'Dirección no disponible'),
        'latitud': e['venue']['location'].get('lat'),
        'longitud': e['venue']['location'].get('lon'),
        'capacidad': e['venue'].get('capacity'),
        'popularidad_score': e.get('score'), 
        'venue_score': e['venue'].get('score') 
    }
    eventos_limpios.append(info)

df = pd.DataFrame(eventos_limpios)

In [32]:
df['capacidad'] = df['capacidad'].replace(0, np.nan)
df = df.drop(["latitud", "longitud"], axis= 1)

In [33]:
df['hora_inicio'] = pd.to_datetime((df['hora_inicio']))
df['hora_inicio'] = df['hora_inicio'].dt.strftime('%H:%M')

In [34]:
tiempos_salida = {
    
    'nba': 2.5, 'nfl': 3.5, 'mlb': 3.0, 'nhl': 2.5, 'mls': 2.0, 
    'ncaa_basketball': 2.5, 'ncaa_football': 3.5, 'sports': 2.5,
    'tennis': 4.0, 'wwe': 3.0, 'boxing': 3.5, 'mma': 3.5,
    
    
    'concert': 3.0, 'music_festival': 8.0, 'classical': 2.5, 'opera': 3.0,
    
    
    'theater': 2.5, 'broadway_tickets_national': 2.5, 'comedy': 2.0, 
    'family': 2.0, 'ballet': 2.5, 'cirque_du_soleil': 2.0
}

In [35]:
def calcular_salida(fila):
    tipo_evento = fila['tipo']
    horas_duracion = tiempos_salida.get(tipo_evento, 2.5) 
    hora_inicio = pd.to_datetime(fila['hora_inicio'])
    hora_fin = hora_inicio + timedelta(hours=horas_duracion)
    
    return hora_fin.strftime('%H:%M')

In [36]:
df['hora_salida_estimada'] = df.apply(calcular_salida, axis=1)


In [37]:
df

,nombre_evento,tipo,hora_inicio,lugar,direccion,capacidad,popularidad_score,venue_score,hora_salida_estimada
0,Wicked - New York,broadway,19:00,Gershwin Theatre,222 West 51st Street,NaN,0.564297,0.76,21:30
1,Romy & Michele: The Musical - New York,theater,19:00,Stage 42,422 West 42nd Street,NaN,0.456264,0.51,21:30
2,The Lion King - New York,broadway,14:00,Minskoff Theatre,200 West 45th Street,NaN,0.489737,0.71,16:30
3,Heathers - New York,theater,19:00,New World Stages - Stage 1,340 West 50th Street,NaN,0.470557,0.64,21:30
4,Two Strangers (Carry a Cake Across New York) -...,broadway,19:30,Longacre Theatre,220 West 48th Street,NaN,0.455934,0.66,22:00
5,The Play That Goes Wrong - New York,theater,19:00,New World Stages - Stage 4,340 West 50th Street,NaN,0.453885,0.66,21:30
6,The Great Gatsby - The Musical - New York,broadway,19:00,Broadway Theatre - New York,1681 Broadway,NaN,0.450506,0.70,21:30
7,Stranger Things: The First Shadow - New York,broadway,13:00,Marquis Theatre - New York,1535 Broadway,NaN,0.423958,0.70,15:30
8,Chicago - The Musical - New York,broadway,19:00,Ambassador Theatre - New York,219 West 49th Street,1155.0,0.411652,0.70,21:30
9,11 to Midnight - New York,theater,14:00,Orpheum Theatre - NY,126 Second Ave,NaN,0.411017,0.54,16:30
